In [ ]:
# 과제 7
import pandas as pd
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM
import numpy as np


data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/hw7_dataset.csv',encoding="ISO-8859-1")

y = data['v1'].map({'spam': 1, 'ham': 0})
x = data['v2']


X_train_val, X_test, y_train_val, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.125, random_state=42)

# Tokenizer 인스턴스를 생성합니다. num_words는 사용할 단어의 최대 개수입니다.
tokenizer = Tokenizer()
tokenizer_val = Tokenizer()
tokenizer_test = Tokenizer()

# 데이터셋의 텍스트를 피팅합니다. 이것은 tokenizer가 단어 인덱스를 구축하게 합니다.
tokenizer.fit_on_texts(X_train)
tokenizer_val.fit_on_texts(X_val)
tokenizer_test.fit_on_texts(X_test)

# 각 문장을 정수 시퀀스로 변환합니다.
sequences = tokenizer.texts_to_sequences(X_train)
sequences_val = tokenizer_val.texts_to_sequences(X_val)
sequences_test = tokenizer_test.texts_to_sequences(X_test)


# 시퀀스를 동일한 길이로 패딩합니다. maxlen은 시퀀스의 최대 길이입니다.
pad_train_data = pad_sequences(sequences, maxlen = 100)
pad_validation_data = pad_sequences(sequences_val, maxlen = 100)
pad_test_data = pad_sequences(sequences_test, maxlen = 100)

word_size = len(tokenizer.word_index) + 1
print(word_size)


model = Sequential()
model.add(Embedding(word_size, 1000, input_length = 100))
model.add(LSTM(64))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
history = model.fit(pad_train_data, y_train, epochs = 200, validation_data  = (pad_validation_data, y_val), batch_size=128, callbacks = [early_stop] )

accuracy = model.evaluate(pad_test_data, y_test)[1]

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


plt.figure(figsize=(12, 4))

# 손실(loss)에 대한 그래프
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 정확도(accuracy)에 대한 그래프
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()



# 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
y_pred = model.predict(pad_test_data)
y_pred = (y_pred > 0.5).astype(int)  # 이진 분류의 경우, 0.5 임계값을 사용하여 레이블을 결정합니다.

# 실제 레이블과 예측 레이블을 비교하여 분류 보고서를 생성합니다.
report = classification_report(y_test, y_pred, output_dict=True)

# 분류 보고서를 DataFrame으로 변환합니다.
df_report = pd.DataFrame(report).transpose()

# 시각화
plt.figure(figsize=(10, 6))
sns.heatmap(df_report, annot=True, fmt=".2f", cmap="Blues")  # 'support' 포함
plt.title("Classification Report")
plt.show()


FileNotFoundError: ignored